<a href="https://colab.research.google.com/github/vedanth-i/483-final-project/blob/main/483_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#
# geometric_libs = '/content/notebooks'
# os.symlink("/content/drive/MyDrive/Colab Notebooks", geometric_libs)
# sys.path.insert(0,geometric_libs)

In [26]:
import os
import torch
print("Using torch", torch.__version__)

Using torch 2.1.0+cu118


In [25]:
!pip install --target=$geometric_libs torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
#can skip this step next time since already installed on drive
#!pip install -r "/content/drive/MyDrive/Colab Notebooks/requirements.txt"

In [27]:
!pip install --target=$geometric_libs ogb
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
# !pip install ogb  # for datasets

In [ ]:
import torch_scatter, torch_sparse, torch_cluster, torch_spline_conv, torch_geometric, ogb

In [ ]:
# import torch_geometric.data into environment
from torch_geometric.data import Data
from torch_geometric import nn
import torch_geometric.transforms as T
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

In [ ]:
from torch_geometric.datasets import Amazon

In [ ]:
dataset_computers = Amazon('/tmp/amazoncomputers', 'Computers')

Processing...
Done!


In [ ]:
dataset_photos = Amazon('/tmp/amazonphotos', 'Photo')

Processing...
Done!


In [ ]:
dataset_computers.num_classes, dataset_photos.num_classes

(10, 8)

In [ ]:
class MLP1(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()

        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        self.layer_2 = nn.Linear(hidden_dim, output_dim)
        self.act = nn.ReLU()


    def forward(self, x):

        x = x.view(1, -1)
        out = 0

        out = self.layer_1(x) #linear 1
        out = self.act(out) #relu
        out = self.layer_2(out) #linear 2

        return out

In [ ]:
#CHANGE HERE these values to get results

hidden_channels = 200 #can change number of hidden channels for variability
l_rate = 0.01 #learning rate
split = 11000 #train/test split.
epochs = 50

In [ ]:
#Computers dataset

num_features_computers = dataset_computers.num_features
num_classes_computers = dataset_computers.num_classes # please write down the number of classes



In [ ]:
#Photos dataset


num_features_photos = dataset_photos.num_features
num_classes_photos = dataset_photos.num_classes # please write down the number of classes



In [ ]:
#define model here

#MLP Model

model_1_computers = MLP1(num_features_computers, hidden_channels, num_classes_computers)
model_1_photos = MLP1(num_features_photos, hidden_channels, num_classes_photos)


In [ ]:
optimizer_1_computers = optim.Adam(model_1_computers.parameters(), lr=l_rate)
optimizer_1_photos = optim.Adam(model_1_photos.parameters(), lr=l_rate) #play around with the learning rate for differentiating

loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(model, data, optimizer, loss_fn):

    loss = 0

    model.train()

    for i in range(0, split): #train/test split is 0.8/0.2. For computers, 0.8*13,752 = 11000. For photos 0.8*7650 = 6048

      optimizer.zero_grad()

      y_pred = model(data.x[i])


      y_actual = torch.zeros(1,10)
      y_actual[0][data.y[i]] = 1

      torch.squeeze(y_pred, dim=0)
      #print(y_pred.shape, data.y.shape, data.y, y_pred)

      l = loss_fn(y_pred, y_actual)  # calculate the loss

      l.backward()  # calculate the gradients of each parameter

      optimizer.step()  # update the parameters by taking an optimizer step

      loss += l.item()


    #########################################

    return loss

In [ ]:
dataset_computers.data, dataset_computers.num_features, dataset_computers.num_classes

/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


(Data(x=[13752, 767], edge_index=[2, 491722], y=[13752]), 767, 10)

In [ ]:
dataset_photos.data, dataset_photos.num_features, dataset_photos.num_classes

(Data(x=[7650, 745], edge_index=[2, 238162], y=[7650]), 745, 8)

In [ ]:
def calculate_accuracy(pred, label):

  acc = 0.0

  new = torch.softmax(pred, dim=1).argmax(dim=1)
  # print(new, new.shape)
  # print(label.shape)
  final = torch.eq(new, label).sum().item()
  acc = final/len(pred)


  return acc

In [ ]:
def evaluate(model, data, loss_fn):

    loss = 0
    epoch_acc = 0


    model.eval()
    for i in range(split, len(data.y)): #0.2 split for testing. Remainder of examples used. For computers, ~2750 examples. For photos ~1512

      y_pred = model(data.x[i])

      y_actual = torch.zeros(1,10)
      y_actual[0][data.y[i]] = 1

      #print(y_pred.shape, data.y.shape, data.y, y_pred)

      l = loss_fn(y_pred, y_actual)
      loss += l.item()
      epoch_acc += calculate_accuracy(y_pred, y_actual)


    return epoch_acc / len(data.y)

In [24]:
best_val_acc = final_test_acc = 0
for epoch in range(1, epochs + 1):
    loss = train(model_1_computers, dataset_computers.data, optimizer_1_computers, loss_fn)
    print(loss)


16454.424340512167
14225.964838509126
13282.32605326739
13633.966436936806
13150.41147286529
12973.924073203078
13570.500383351022
12340.400008688783
12697.82826375557
13286.450244209776
12227.133865755079
12469.326646800439
10844.910644144156
11521.522878531892
12186.617009594973
12285.837621108514
12262.486194462566
11641.228814880576
11096.556137154694
10793.456572018673
11712.999302335691
11661.830749156434
11086.151141838516


KeyboardInterrupt: ignored

In [28]:
#Save model's parameters like such. Can change parameters depending on what model you want to save
torch.save(model_1_photos.state_dict(), 'model_photos_200_0.001.pt')  # saving model's parameters

In [30]:
test_acc = evaluate(model_1_computers, dataset_computers.data, loss_fn)
print(test_acc)

In [31]:
from torch_geometric.nn import GCNConv

class GCN1(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)

        self.conv3 = GCNConv(hidden_channels, hidden_channels)

        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.act = torch.nn.ReLU()

    def forward(self, node_feature, edge_index):

        output = self.conv1(node_feature, edge_index)
        output = self.act(output)

        #Can add/remove GCN layers

        # output = self.conv3(output, edge_index)
        # output = self.act(output)
        # output = self.conv3(output, edge_index)
        # output = self.act(output)

        # output = self.conv3(output, edge_index)
        # output = self.act(output)
        # output = self.conv3(output, edge_index)
        # output = self.act(output)

        # output = self.conv3(output, edge_index)
        # output = self.act(output)
        # output = self.conv3(output, edge_index)
        # output = self.act(output)


        output = self.conv2(output, edge_index)

        return output

In [32]:
#CHANGE HERE these values to get results

hidden_channels = 200 #can change number of hidden channels for variability
l_rate = 0.01 #learning rate
split = 11000 #train/test split
epochs = 50

In [33]:

#Computers and Photos

num_features_computers = dataset_computers.num_features
num_classes_computers = dataset_computers.num_classes # please write down the number of classes

num_features_photos = dataset_photos.num_features
num_classes_photos = dataset_photos.num_classes # please write down the number of classes



In [34]:
#GCN Models
gcn_computers = GCN1(num_features_computers, hidden_channels, num_classes_computers)
gcn_photos = GCN1(num_features_photos, hidden_channels, num_classes_photos)

In [35]:
#GCN Optimizers
gcn_opt_computers = optim.Adam(gcn_computers.parameters(), lr=l_rate)
gcn_opt_photos = optim.Adam(gcn_photos.parameters(), lr=l_rate) #play around with the learning rate for differentiating

In [36]:
loss_fn = nn.CrossEntropyLoss()

In [37]:
def trainGCN(model, data, optimizer, loss_fn):

    loss = 0

    model.train()

    optimizer.zero_grad()

    y_pred = model(data.x, data.edge_index)

    l = loss_fn(y_pred[:split], data.y.view(-1)[:split])  # calculate the loss

    l.backward()  # calculate the gradients of each parameter

    optimizer.step()  # update the parameters by taking an optimizer step

    loss += l.item()


    #########################################

    return loss

In [38]:
def testGCN(model, data):


  acc = 0
  model.eval()
  pred = model(data.x, data.edge_index)
  print(pred.shape)

  predicted_classes = torch.argmax(pred[split:], dim=1)
  acc = torch.mean((predicted_classes == data.y[split:]).float()).item()

  #########################################

  return acc


In [39]:
for epoch in range(1, epochs + 1):
    loss = trainGCN(gcn_computers, dataset_computers.data, gcn_opt_computers, loss_fn)
    print(loss)

2.2540762424468994
9.513840675354004
9.907833099365234
6.928719520568848
4.022804260253906
2.996025323867798
2.318814516067505
2.543565034866333
2.425250768661499
2.1283323764801025
1.9522950649261475
1.9743455648422241
1.934085726737976
1.771753191947937
1.6756209135055542
1.6685841083526611
1.7079312801361084
1.7113721370697021
1.632947564125061
1.539300560951233
1.490108847618103
1.4760775566101074
1.4591832160949707
1.4231340885162354
1.3625049591064453
1.3452816009521484
1.2860833406448364
1.2724738121032715
1.2334543466567993
1.213301181793213
1.2233988046646118
1.1295855045318604
1.0921144485473633
1.074951171875
1.0534697771072388
1.033928394317627
1.0026543140411377
0.967704713344574
0.9404873251914978
0.920881986618042
0.9030722379684448
0.8816046714782715
0.878686249256134
0.8373917937278748
0.828613817691803
0.810575544834137
0.7872129082679749
0.7694396376609802
0.7542781233787537
0.7386355996131897


In [40]:
torch.save(gcn_computers.state_dict(), 'gcn_computers_layers.pt')  # saving model's parameters

In [41]:
test_acc = testGCN(gcn_computers, dataset_computers.data)
print(test_acc)

torch.Size([13752, 10])
0.7910610437393188
